# 🖼️ 1. Datensatz vorbereiten

Jede Gruppe erhält einen Ordner mit **Rohbildern**. Diese sollen in [Makesense.ai](https://www.makesense.ai) annotiert werden.

## 🔧 Schritt 1: Annotation mit Makesense.ai
0. Ladet die Rohbilder mit eurer Gruppennummer unter folgender [URL](https://databox.bonn.de/public/upload-shares/YhXlVGe57thQAg8WUjVlZFNK6z4LORiq) runter
1. Öffnet [https://www.makesense.ai](https://www.makesense.ai)
2. Bilder hochladen
3. Modus: **Segmentation**
4. Klasse: z. B. `baum`, `überdachung`
5. Export: Format `Pascal VOC Segmentation (PNG masks)`

👉 Jetzt bitte eure annotierten Masken und Bilder wieder hochladen. Verwende folgenden Platzhalter:


In [ ]:
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
import os

# Für stabile Anzeige in JupyterLab
from IPython.display import display

# Arbeitsverzeichnis
annotations_file = "datensatz_gruppe1/annotations.json"
images_dir = "datensatz_gruppe1"

# COCO laden
coco = COCO(annotations_file)
image_ids = coco.getImgIds()

# Nimm das erste Bild
img_id = image_ids[0]
img_info = coco.loadImgs(img_id)[0]
img_path = os.path.join(images_dir, img_info['file_name'])

# Bild laden
image = Image.open(img_path)

# Plot vorbereiten
fig = plt.figure(figsize=(8, 8))
plt.imshow(image)
ax = plt.gca()

# Annotations laden
ann_ids = coco.getAnnIds(imgIds=img_id)
anns = coco.loadAnns(ann_ids)

# Maske zeichnen
for ann in anns:
    if 'segmentation' in ann and isinstance(ann['segmentation'], list):
        for seg in ann['segmentation']:
            poly = np.array(seg).reshape((len(seg) // 2, 2))
            patch = patches.Polygon(poly, fill=False, edgecolor='red', linewidth=2)
            ax.add_patch(patch)

plt.title(img_info['file_name'])
plt.axis('off')

: 

## 📈 Datensatz-Erweiterung durch Augmentation

Um die Robustheit und Generalisierungsfähigkeit des Modells zu verbessern, wurde der Datensatz durch gezielte Bildaugmentierung erweitert. Dabei wurde zu jedem bestehenden Bild **eine sinnvolle Transformation** erzeugt.

Die angewendeten Augmentierungsschritte umfassen:
- **Horizontales Spiegeln**
- **Leichte Rotation (±15°)**
- **Änderung von Helligkeit und Kontrast**

Für jedes Originalbild wurde ein neues Bild generiert. Die zugehörigen Segmentierungs-Masken wurden dabei korrekt mittransformiert. Die augmentierten Bilder und Masken wurden zusammen mit den Originalen in einer neuen COCO-kompatib


In [ ]:
import albumentations as A
import cv2
import json
import os
import numpy as np
from pycocotools import mask as mask_utils
import uuid
from tqdm import tqdm

# 📂 Verzeichnisse
source_dir = "datensatz_gruppe1"
target_dir = "datensatz_gruppe1_augmented"
os.makedirs(target_dir, exist_ok=True)

# 📖 Originaldaten laden
with open(os.path.join(source_dir, "annotations.json"), "r") as f:
    coco_data = json.load(f)

# Neue COCO-Struktur
aug_data = {
    "images": [],
    "annotations": [],
    "categories": coco_data["categories"]
}

# ID-Offsets
next_image_id = 10000
next_ann_id = 50000

# Bild-ID → Annotations
img_to_anns = {}
for ann in coco_data["annotations"]:
    img_to_anns.setdefault(ann["image_id"], []).append(ann)

# ✅ Nur sichere Augmentierungen (keine schwarzen Ränder)
transform_options = [
    ("flipH", A.HorizontalFlip(p=1.0)),
    ("flipV", A.VerticalFlip(p=1.0)),
    ("rot90", A.RandomRotate90(p=1.0))
]

for img_entry in tqdm(coco_data["images"], desc="Augmentiere Bilder"):
    file_name = img_entry["file_name"]
    img_path = os.path.join(source_dir, file_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    anns = img_to_anns.get(img_entry["id"], [])
    if not anns:
        continue

    # Segmentierungen → Masken
    masks = []
    for ann in anns:
        for seg in ann["segmentation"]:
            pts = np.array(seg).reshape(-1, 2)
            mask = np.zeros(img.shape[:2], dtype=np.uint8)
            cv2.fillPoly(mask, [pts.astype(np.int32)], 1)
            masks.append(mask)

    # ✅ Augmentation auswählen
    aug_name, transform = A.OneOf([
        (A.HorizontalFlip(p=1.0), 1),
        (A.VerticalFlip(p=1.0), 1),
        (A.RandomRotate90(p=1.0), 1),
    ], p=1.0)(image=img, masks=masks).items()


## Visualisierung der Augmentation

In [ ]:
import os
import json

# 🔧 Pfade anpassen
image_dir = "datensatz_gruppe1_extended"
json_path = os.path.join(image_dir, "annotations_extended.json")

# 📖 JSON laden
with open(json_path, "r") as f:
    data = json.load(f)

# 🔎 Bildnamen aus JSON
json_images = {img["file_name"] for img in data["images"]}
json_image_ids = {img["id"] for img in data["images"]}

# 🔎 Bilddateien im Ordner
folder_images = {f for f in os.listdir(image_dir) if f.lower().endswith((".jpg", ".png"))}

# 🔎 IDs aus Annotations
referenced_ids = {ann["image_id"] for ann in data["annotations"]}

# ✅ Checks
missing_files = json_images - folder_images
unreferenced_files = folder_images - json_images
annotations_without_images = referenced_ids - json_image_ids

# 🖨️ Ergebnis
print("📁 Bilder in JSON, aber nicht im Ordner:", missing_files if missing_files else "✅ Keine")
print("📁 Bilder im Ordner, aber nicht in JSON:", unreferenced_files if unreferenced_files else "✅ Keine")
print("📛 Annotations mit fehlenden Bild-IDs:", annotations_without_images if annotations_without_images else "✅ Keine")


## Datensatz prüfen

In [ ]:
import os
import json

# 🔧 Pfade anpassen
image_dir = "datensatz_gruppe1_extended"
json_path = os.path.join(image_dir, "annotations_extended.json")

# 📖 JSON laden
with open(json_path, "r") as f:
    data = json.load(f)

# 🔎 Bildnamen aus JSON
json_images = {img["file_name"] for img in data["images"]}
json_image_ids = {img["id"] for img in data["images"]}

# 🔎 Bilddateien im Ordner
folder_images = {f for f in os.listdir(image_dir) if f.lower().endswith((".jpg", ".png"))}

# 🔎 IDs aus Annotations
referenced_ids = {ann["image_id"] for ann in data["annotations"]}

# ✅ Checks
missing_files = json_images - folder_images
unreferenced_files = folder_images - json_images
annotations_without_images = referenced_ids - json_image_ids

# 🖨️ Ergebnis
print("📁 Bilder in JSON, aber nicht im Ordner:", missing_files if missing_files else "✅ Keine")
print("📁 Bilder im Ordner, aber nicht in JSON:", unreferenced_files if unreferenced_files else "✅ Keine")
print("📛 Annotations mit fehlenden Bild-IDs:", annotations_without_images if annotations_without_images else "✅ Keine")
